# QuestDB Connect and Query

## Querying QuestDB using the PostgreSQL wire-protocol

QuestDB is compatible with the Postgresql wire-protocol, which means you can use any postgresql compatible library to connect, send queries, and get results back. Please note that QuestDB is not built on top of Postgresql and it doesn't support its whole function set. 

Moreover, QuestDB provides [its own extensions](https://questdb.io/docs/concept/sql-extensions/) to work with time-series data. However, at the protocol level the SQL is not validated, so as long as your Postgresql client can connect, send queries, and read results back, you can use it to query QuestDB.

In [1]:
import psycopg as pg
import time
import os

# Fetch environment variables with defaults
host = os.getenv('QDB_CLIENT_HOST', 'questdb')
port = os.getenv('QDB_CLIENT_PORT', '8812')
user = os.getenv('QDB_CLIENT_USER', 'admin')
password = os.getenv('QDB_CLIENT_PASSWORD', 'quest')

# Create the connection string using the environment variables or defaults
conn_str = f'user={user} password={password} host={host} port={port} dbname=qdb'
conn_str = 'user=admin password=quest host=questdb port=8812 dbname=qdb'
with pg.connect(conn_str, autocommit=True) as connection:

    # Open a cursor to perform database operations

    with connection.cursor() as cur:

        #Query the database and obtain data as Python objects.

        cur.execute('SELECT * FROM github_events LIMIT 5;')
        records = cur.fetchall()
        for row in records:
            print(row)

# the connection is now closed


('AadityaJain17/jobgenie', 'CreateEvent', 'AadityaJain17', datetime.datetime(2024, 9, 24, 17, 25, 58, 179036))
('trafver54336457645/trafver54336457645', 'PushEvent', 'github-actions[bot]', datetime.datetime(2024, 9, 24, 17, 25, 58, 181431))
('goofysun/Get-IP-SELF', 'PushEvent', 'github-actions[bot]', datetime.datetime(2024, 9, 24, 17, 25, 58, 181474))
('kakao-travel-mandi/mandi-backend', 'PullRequestEvent', 'LEE-sh1673', datetime.datetime(2024, 9, 24, 17, 25, 58, 181521))
('integration-os/integrationos', 'PushEvent', 'paulkr', datetime.datetime(2024, 9, 24, 17, 25, 58, 181549))


## Querying QuestDB using the REST API

It is possible to use QuestDB's API from any HTTP client. As an example we will use the Python `requests` library to issue the same query we executed above

In [5]:
import requests

host = 'http://questdb:9000'

sql_query = 'SELECT * FROM github_events LIMIT 5;'

try:
    response = requests.get(
        host + '/exec',
        params={'query': sql_query}).json()
    for row in response['dataset']:
        print(row)
except requests.exceptions.RequestException as e:
    print(f'Error: {e}')

['cs-jsi/chisel4ml', 'PushEvent', 'jurevreca12', '2024-02-02T08:05:12.023344Z']
['Tarzan22/BES-Backend', 'CreateEvent', 'Tarzan22', '2024-02-02T08:05:12.026647Z']
['haoqing0110/addon-framework', 'CreateEvent', 'haoqing0110', '2024-02-02T08:05:12.026703Z']
['negoxa2020/web_individ', 'PushEvent', 'negoxa2020', '2024-02-02T08:05:12.026730Z']
['estevenson9/sketch', 'PushEvent', 'estevenson9', '2024-02-02T08:05:12.026751Z']


## Querying QuestDB using the REST API from the command line

You can of course use any command line tools, like `curl` to execute queries on questdb. This also allows you to run DDL statements and execute some admin tasks


In [8]:
!curl -G --silent --data-urlencode "query=SELECT * FROM github_events LIMIT 5;"  http://questdb:9000/exec | json_pp

{
   "columns" : [
      {
         "name" : "repo",
         "type" : "SYMBOL"
      },
      {
         "name" : "type",
         "type" : "SYMBOL"
      },
      {
         "name" : "actor",
         "type" : "STRING"
      },
      {
         "name" : "timestamp",
         "type" : "TIMESTAMP"
      }
   ],
   "count" : 5,
   "dataset" : [
      [
         "cs-jsi/chisel4ml",
         "PushEvent",
         "jurevreca12",
         "2024-02-02T08:05:12.023344Z"
      ],
      [
         "Tarzan22/BES-Backend",
         "CreateEvent",
         "Tarzan22",
         "2024-02-02T08:05:12.026647Z"
      ],
      [
         "haoqing0110/addon-framework",
         "CreateEvent",
         "haoqing0110",
         "2024-02-02T08:05:12.026703Z"
      ],
      [
         "negoxa2020/web_individ",
         "PushEvent",
         "negoxa2020",
         "2024-02-02T08:05:12.026730Z"
      ],
      [
         "estevenson9/sketch",
         "PushEvent",
         "estevenson9",
         "2024-02-02T08: